![](https://img.shields.io/badge/PO.DAAC-Contribution-%20?color=grey&labelColor=blue)

> From the PO.DAAC Cookbook, to access the GitHub version of the notebook, follow [this link](https://github.com/podaac/tutorials/blob/master/notebooks/SearchDownload_SWOTviaCMR.ipynb).

# Search and Download SWOT Data via `earthaccess`
#### *Author: Cassandra Nickles, PO.DAAC*

## Summary
This notebook will find and download SWOT hydrology data (vD) programmatically via earthaccess python library. For more information about earthaccess visit: https://nsidc.github.io/earthaccess/

## Requirements
### 1. Compute environment 
This tutorial can be run in the following environments:
- **Local compute environment** e.g. laptop, server: this tutorial can be run on your local machine

### 2. Earthdata Login

An Earthdata Login account is required to access data, as well as discover restricted data, from the NASA Earthdata system. Thus, to access NASA data, you need Earthdata Login. Please visit https://urs.earthdata.nasa.gov to register and manage your Earthdata Login account. This account is free to create and only takes a moment to set up.

### Import libraries

In [1]:
import geopandas as gpd
import glob
from pathlib import Path
import pandas as pd
import os
import zipfile
import earthaccess

In this notebook, we will be calling the authentication in the below cell.

In [2]:
auth = earthaccess.login() 

### Search for SWOT data links
We want to find the SWOT files for a particular pass over North America. 

Each dataset has it's own unique shortname, which can be used to search earthaccess. Shortnames can be found on dataset landing pages or [Earthdata Search](https://search.earthdata.nasa.gov/search) Collections.

#### SWOT Level 2 KaRIn High Rate Version D Datasets:

- **Water Mask Pixel Cloud NetCDF** - SWOT_L2_HR_PIXC_D
- **Water Mask Pixel Cloud Vector Attribute NetCDF** - SWOT_L2_HR_PIXCVec_D
- **River Vector Shapefile** - SWOT_L2_HR_RiverSP_D
- **Lake Vector Shapefile** - SWOT_L2_HR_LakeSP_D
- **Raster NetCDF** - SWOT_L2_HR_Raster_D

Let's start our search for River Vector Shapefiles with a particular pass, pass 013. SWOT files come in "reach" and "node" versions in the same collection, here we want the 10km reaches rather than the nodes. We will also only get files for North America, or 'NA' and call out a specific pass number that we want.

In [3]:
results = earthaccess.search_data(short_name = 'SWOT_L2_HR_RIVERSP_D', 
                                  #temporal = ('2025-05-05 00:00:00', '2025-05-29 23:59:59'), # can also specify by time
                                  granule_name = '*Reach*_013_NA*') # here we filter by Reach files (not node), pass=013, continent code=NA

granule_list = list(results)
print(f"Total granules found: {len(granule_list)}")

Total granules found: 1


During the science orbit, a pass will be repeated once every 21 days. A particular location may have different passes observe it within the 21 days, however.

### Download the Data into a folder

In [4]:
earthaccess.download(results, "./datasets/data_downloads/SWOT_files/")
folder = Path("./datasets/data_downloads/SWOT_files")

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

### Shapefiles come in a .zip format, and need to be unzipped in the existing folder

In [5]:
for item in os.listdir(folder): # loop through items in dir
    if item.endswith(".zip"): # check for ".zip" extension
        zip_ref = zipfile.ZipFile(f"{folder}/{item}") # create zipfile object
        zip_ref.extractall(folder) # extract file to dir
        zip_ref.close() # close file

In [6]:
os.listdir(folder)

['SWOT_L2_HR_RiverSP_Reach_033_013_NA_20250519T083655_20250519T083727_PID0_01.dbf',
 'SWOT_L2_HR_RiverSP_Reach_033_013_NA_20250519T083655_20250519T083727_PID0_01.prj',
 'SWOT_L2_HR_RiverSP_Reach_033_013_NA_20250519T083655_20250519T083727_PID0_01.shp',
 'SWOT_L2_HR_RiverSP_Reach_033_013_NA_20250519T083655_20250519T083727_PID0_01.shp.xml',
 'SWOT_L2_HR_RiverSP_Reach_033_013_NA_20250519T083655_20250519T083727_PID0_01.shx',
 'SWOT_L2_HR_RiverSP_Reach_033_013_NA_20250519T083655_20250519T083727_PID0_01.zip']